In [11]:
import numpy as np
import math 
from scipy.optimize import fsolve # this is what we going to use to solve this x value
import pandas as pd
import matplotlib.pyplot as plt



# this is some dictionary that we are going to use for the cost and price of the chemicals

cost_adjusted = {
    'EB': 117.03 , # $/kmol 
    'steam' : 1.75, # $/kmol
    
}

adjusted_price = {
    'styrene': 142.33, # $/kmol
    'toluene': 89.38, # $/kmol
    'dihydrogen': 0.93, # $/kmol
    'methane' : 3.04, # $/kmol

}

# it multiplies the price by 2.2046 to convert it to kg, and then divides it by 1000 to convert it to price/g

# this will be used for the conversion of the moles to the mass


 # mol/hr of styrene that we need
# the pressure is going to be in bar, the ratio is going to be described as 
# moles of steam = R*moles of EB
def P_T_R_function(P,T,R,F): # this is the main function im going to useing for this process
    Keq_T = np.exp(-14852.6/T + 15.5408) # temperature dependent equilibrium constant
    
    if T == 800: 
        Selectivity = 0.01

    elif T == 850:
        Selectivity = 0.03
    elif T == 900:
        Selectivity = 0.06
    elif T == 950:
        Selectivity = 0.13
    else:
        print("Temperature is not supported")
    EB0 = 0.95*F # kmol/hr
    T0 =  0.03*F # kmol/hr
    B0 = 0.02*F # kmol/hr    
    def equilibrium_equation(theta_1): # this is the equation that we are going to solve to get the x value


        total_steam_moles =  R*EB0 
        EB = EB0 - theta_1 - (Selectivity*theta_1)/(1-Selectivity) # the amount of Ethylene benzene remaining kmol/hour
        S =  theta_1 # styrene produced kmoles/ hour
        Tol = T0 + (Selectivity*theta_1)/(1-Selectivity)  # the amount of Toluene produced kmoles/hour
        H = theta_1 - (Selectivity*theta_1)/(1-Selectivity)# the amount of Hydrogen produced kmoles/hour
        M = (Selectivity*theta_1)/(1-Selectivity)# the amount of Methane produced kmoles/hour
        n_total = total_steam_moles + EB + S + Tol + H + B0 + T0 + M

        Keq_P = P*(S*H)/(n_total* EB)



        
        return Keq_T - Keq_P # to make sure that thi sis equal to zero 
    # this is the initial guess for the theta value

    Theta1_solution = fsolve(equilibrium_equation,50)[0] # this is the intial guess for the theta value
    Theta1_solution_actual = Theta1_solution * 0.8 # this is the actual value of the theta value
    if Theta1_solution_actual != 96.02: #kmol/hr 
        F_new = 96.02/(Theta1_solution_actual) * F
        theta_new = 96.02 
        
    if not np.isfinite(Theta1_solution): # github helper given assistance 
        print("Solver couldnt converge ")   
        return 0
    return theta_new,F_new # this is the conversion of the EB to styrene and toluene
print(P_T_R_function(1.4,950,12,100)) # this is the test for the function

(96.02, 154.91368443549297)


In [21]:
def Price_Calculator(P,T,R):
    theta_1, F_out = P_T_R_function(P,T,R,100)
    EB0 = 0.95*F_out # kmol/hr
    T0 =  0.03*F_out # kmol/hr
    B0 = 0.02*F_out # kmol/hr 
    if T == 800: 
        Selectivity = 0.01

    elif T == 850:
        Selectivity = 0.03
    elif T == 900:
        Selectivity = 0.06
    elif T == 950:
        Selectivity = 0.13
    else:
        print("Temperature is not supported")

    total_steam_moles =  R*EB0 
    EB = EB0 - theta_1 - (Selectivity*theta_1)/(1-Selectivity) # the amount of Ethylene benzene remaining kmol/hour
    S =  theta_1 # styrene produced kmoles/ hour
    Tol = T0 + (Selectivity*theta_1)/(1-Selectivity)  # the amount of Toluene produced kmoles/hour
    H = theta_1 - (Selectivity*theta_1)/(1-Selectivity)# the amount of Hydrogen produced kmoles/hour
    M = (Selectivity*theta_1)/(1-Selectivity)# the amount of Methane produced kmoles/hour
    n_total = total_steam_moles + EB + S + Tol + H + B0 + T0 + M
    EB_consumed = EB0 - EB
    revenue =(
            S*adjusted_price['styrene'] + 
            Tol*adjusted_price['toluene'] + 
            H*adjusted_price['dihydrogen'] + 
            M*adjusted_price['methane']) # $/hours 
    cost =( 
            EB_consumed*cost_adjusted['EB'] + 
           total_steam_moles*cost_adjusted['steam'] 
           ) # $/hours
    profit = revenue - cost
    return profit
#=======================================================================================================
best_profit = -np.inf
best_condition= [None,None,None]



P_guess = np.linspace(0.4,1.4, num = 10) # this is in bar(0.4-1.6)
T_guess = np.linspace(800, 950, num = 4) # this is in K (800-950)
R_guess = np.linspace(6,12, num= 10) # this is the ratio of steam to EB (6-12)

# this is the initial guess for the P, T, R values to check my answer
P_cool= 1.4
T_cool = 950
R_cool = 12
Feed_cool = 154.91368444
theta_solution_test = P_T_R_function(P_cool,T_cool,R_cool,Feed_cool)
profit_solution_test = Price_Calculator(P_cool,T_cool,R_cool)


print(f'this is the conversion we got at P =1.4, T= 950, and R =12, and Theta1 = {theta_solution_test[0]} and the profit was observed at {profit_solution_test}')
for tempK in T_guess:
    for P in P_guess:
        for R in R_guess:
            profit = Price_Calculator(P, tempK, R)
            if profit is not None and profit > best_profit:
                best_profit    = profit
                best_condition = (P, tempK, R)
                 
if best_condition[0] is None:
    print("No valid (P,T,R) combination found.")
    
else:

    print(f"""
Best Profit:          {best_profit:.3f} $/hour
Best Pressure (bar):  {best_condition[0]:.3f}
Best Temperature (K): {best_condition[1]:.3f}
Best Steam Ratio:     {best_condition[2]:.3f}
""")

this is the conversion we got at P =1.4, T= 950, and R =12, and Theta1 = 96.02 and the profit was observed at -522.9810740312569

Best Profit:          2664.713 $/hour
Best Pressure (bar):  0.400
Best Temperature (K): 900.000
Best Steam Ratio:     12.000



In [24]:
results = []

for tempK in T_guess:
    profits = []
    pressures = []
    steam_ratios = []

    for P in P_guess:
        for R in R_guess:
            profit = Price_Calculator(P, tempK, R)
            profits.append(profit)
            pressures.append(P)
            steam_ratios.append(R)

    df = pd.DataFrame({
        "Pressure (bar)": pressures,
        "Steam Ratio": steam_ratios,
        "Profit ($/hour)": profits
    })
    results.append((tempK, df))

# Specify a valid path or just use a filename in the current directory:
excel_filename = "profit_optimization.xlsx"

with pd.ExcelWriter(excel_filename) as writer:
    for tempK, df in results:
        df.to_excel(writer, sheet_name=f"T_{int(tempK)}K", index=False)

print(f"Data successfully saved to {excel_filename}.")

Data successfully saved to profit_optimization.xlsx.


In [26]:
def get_streams(P, T, R): # github helper assistance 
    """
    Return a dictionary of all the inlet and outlet flows for given (P, T, R, F).
    """
    theta_1, F_out = P_T_R_function(P, T, R, 100)

    EB0 = 0.95 * F_out
    T0  = 0.03 * F_out
    B0  = 0.02 * F_out

    # Determine selectivity
    if T == 800:
        Selectivity = 0.01
    elif T == 850:
        Selectivity = 0.03
    elif T == 900:
        Selectivity = 0.06
    elif T == 950:
        Selectivity = 0.13
    else:
        raise ValueError("Unsupported temperature") # github helper assistance 

    total_steam_moles = R * EB0
    EB_out = EB0 - theta_1 - (Selectivity*theta_1)/(1-Selectivity)
    S      = theta_1
    Tol    = T0 + (Selectivity*theta_1)/(1-Selectivity)
    H      = theta_1 - (Selectivity*theta_1)/(1-Selectivity)
    M      = (Selectivity*theta_1)/(1-Selectivity)
    n_total = total_steam_moles + EB_out + S + Tol + H + B0 + T0 + M

    # Return all streams in a dictionary
    return {
        "F_out" : F_out,
        "EB0"   : EB0,
        "T0"    : T0,
        "B0"    : B0,
        "steam_in": total_steam_moles,
        "EB_out": EB_out,
        "S"     : S,
        "Tol"   : Tol,
        "H"     : H,
        "M"     : M,
        "n_total": n_total
    }

# Then, you can do
streams = get_streams(P_cool, T_cool, R_cool) # github helper assistance 
print("===== STREAM INFORMATION =====")
for k, v in streams.items():
    print(f"{k}: {v}")


===== STREAM INFORMATION =====
F_out: 154.91368443549297
EB0: 147.1680002137183
T0: 4.647410533064789
B0: 3.0982736887098596
steam_in: 1766.0160025646196
EB_out: 36.80018412176428
S: 96.02
Tol: 18.995226625018812
H: 81.67218390804598
M: 14.347816091954023
n_total: 2021.5970975331775
